# Paths

In [1]:
path = '/home/chayan/NeuroGDAVIS/'
res_plots = path + 'Results/Plots/'
path_emb = path + 'Data/Embeddings/'

# Importing libraries

In [2]:
import sys
sys.path.append(path)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import keras
import ipynbname
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from utils import Datasets as ds
from keras.models import load_model
from utils import DownstreamAnalysis as DA

dataname = ipynbname.name().rsplit("_")[0]
txt = %pwd
methodname = txt.rsplit("/")[-1]

# Load labels and embeddings

In [3]:
y = ds.LoadData(dataname, DType='None', LType=True)
x_embedding = np.array(pd.read_csv(path_emb+dataname+'/'+dataname+'_'+methodname+'.csv', index_col=0, header=0))

# Downstream analysis

### k-NN

In [4]:
neighbour = [5, 15, 25, 35, 45]
Score_knn = []
for n in neighbour:
    Score_knn.append(DA.Knn(x_embedding, y, n))
Score_knn = pd.DataFrame(Score_knn, index=['neighbour '+ str(s) for s in neighbour],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_knn

,Accuracy,Precision,Recall,F1-Score
neighbour 5,0.7205,0.717179,0.7205,0.716212
neighbour 15,0.7310,0.728280,0.7310,0.728323
neighbour 25,0.7275,0.726487,0.7275,0.725710
neighbour 35,0.7280,0.726283,0.7280,0.725549
neighbour 45,0.7245,0.721680,0.7245,0.721221


### Random Forest

In [5]:
n_tree = [20,40,60,80,100]
Score_rfc = []
for n in n_tree:
    Score_rfc.append(DA.RFC(x_embedding, y, n))
Score_rfc = pd.DataFrame(Score_rfc, index=['n_tree '+ str(s) for s in neighbour],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_rfc

,Accuracy,Precision,Recall,F1-Score
n_tree 5,0.7145,0.713993,0.7145,0.713272
n_tree 15,0.7210,0.720161,0.7210,0.720055
n_tree 25,0.7140,0.713133,0.7140,0.712727
n_tree 35,0.7100,0.709748,0.7100,0.709254
n_tree 45,0.7150,0.714360,0.7150,0.714100


### k-Means

In [6]:
iteration = 10
Results_km = []

for i in tqdm(range(iteration), desc='Progress'):
    Results_km.append(DA.kmeans(x_embedding, y))
    
Results_km = pd.DataFrame(Results_km,columns = ['ARI', 'FMI','NMI'])
Results_km

Progress: 100%|█████████████████████████████████| 10/10 [00:00<00:00, 12.24it/s]


,ARI,FMI,NMI
0,0.240214,0.426282,0.325654
1,0.260403,0.439101,0.340329
2,0.243671,0.420773,0.323180
3,0.276036,0.451929,0.353244
4,0.248868,0.431253,0.329191
5,0.224890,0.401999,0.308166
6,0.258402,0.437288,0.338456
7,0.276626,0.452197,0.353635
8,0.260356,0.440675,0.341702
9,0.276421,0.467790,0.354893


### Agglomerative 

In [7]:
idx = ['ARI', 'FMI','NMI']
AglScore = pd.DataFrame(DA.Agglomerative(x_embedding, y), index = idx)    #seurat_clusters, Cluster, celltype.l2
AglScore.T

,ARI,FMI,NMI
0,0.23978,0.428219,0.323109
